In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy  as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/additional-kiva-snapshot/loans_lenders.csv
/kaggle/input/additional-kiva-snapshot/gmaps.png
/kaggle/input/additional-kiva-snapshot/KadunaNigeria.png
/kaggle/input/additional-kiva-snapshot/loans.csv
/kaggle/input/additional-kiva-snapshot/country_stats.csv
/kaggle/input/additional-kiva-snapshot/GEconV4.csv
/kaggle/input/additional-kiva-snapshot/cover_small.png
/kaggle/input/additional-kiva-snapshot/locations.csv
/kaggle/input/additional-kiva-snapshot/cover_v2.png
/kaggle/input/additional-kiva-snapshot/lenders.csv
/kaggle/input/additional-kiva-snapshot/Leaderboard.png
/kaggle/input/additional-kiva-snapshot/loan_coords.csv


In [2]:
import random
#import time

1. Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

In [3]:
#carico
loan_lenders = pd.read_csv("../input/additional-kiva-snapshot/loans_lenders.csv")
loan_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [4]:
#splitto il campo lenders dove c'è la virgola
loan_lenders['lenders'] = loan_lenders.lenders.str.split(',')

In [5]:
#rendo righe separate tutti i lenders divisi nel punto precedente
df_lenders = loan_lenders.explode('lenders')
df_lenders.head()

,loan_id,lenders
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499


In [6]:
#doppioni

2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [7]:
#carico
loans = pd.read_csv("../input/additional-kiva-snapshot/loans.csv")

In [8]:
#rendo da stringa a datetime il formato
loans['planned_expiration_time'] = pd.to_datetime(loans['planned_expiration_time']) 
loans['disburse_time']           = pd.to_datetime(loans['disburse_time'])

#calcolo la differenza in giorni tra i due campi prendendo 
loans['duration'] = loans['planned_expiration_time'].dt.date -\
                    loans['disburse_time'].dt.date

loans[['loan_id','loan_name','duration']].head()

,loan_id,loan_name,duration
0,657307,Aivy,54 days
1,657259,Idalia Marizza,96 days
2,658010,Aasia,37 days
3,659347,Gulmira,35 days
4,656933,Ricky\t,58 days


Controllo i missing

In [9]:
#conto quanti sono nei tre campi usati
#loans[['disburse_time','planned_expiration_time','duration']].isnull().sum()

In [10]:
#prendo le row che hanno valori nulli per disburse_time e guardo quante sono nulle
#nello stesso tempo anche per planned_expiration_time
loans[loans[['disburse_time']].isna().any(axis=1)]\
['planned_expiration_time'].isnull().sum()

2

In [11]:
#controllo che i missing siano giusti 
#(dalla somma devo togliere le due righe che sono nulle per tutti e due)
if loans['disburse_time'].isnull().sum() + loans['planned_expiration_time'].isnull().sum() - 2 == loans['duration'].isnull().sum():
    print('OK :', loans['disburse_time'].isnull().sum() ,'+', loans['planned_expiration_time'].isnull().sum(),'- 2','=', loans['duration'].isnull().sum())
else:
    print('ERRORE')

OK : 2813 + 371834 - 2 = 374645


3. Find the lenders that have funded at least twice

In [12]:
#prendo i lenders, conto quante volte compaiono, prendo solo quelli che compaiono 2 volte
lenders_2 = df_lenders.lenders\
                      .value_counts()\
                      .loc[lambda x: x >= 2]\
                      .reset_index()
lenders_2

,index,lenders
0,gooddogg1,151307
1,trolltech4460,148347
2,gmct,127089
3,nms,100360
4,themissionbeltco,76986
...,...,...
1005106,kim5468,2
1005107,lilly1361,2
1005108,marylou6056,2
1005109,rajesh4225,2


4. For each country, compute how many loans have involved that country as borrowers

In [13]:
#controllo quanti missing ci sono in country code
print(loans['country_code'].isnull().sum())
#guardo come sono fatti
loans[loans[['country_code']].isna().any(axis=1)]['country_name']

9


82889      Namibia
156970     Namibia
598087     Namibia
684876     Namibia
971827     Namibia
1134818    Namibia
1214923    Namibia
1281022    Namibia
1415763    Namibia
Name: country_name, dtype: object

In [14]:
#controllo che non siano presenti righe che abbiano il country_name trovato prima e un country_code diverso da NaN
loans.loc[loans["country_name"] == "Namibia"]["country_code"]

82889      NaN
156970     NaN
598087     NaN
684876     NaN
971827     NaN
1134818    NaN
1214923    NaN
1281022    NaN
1415763    NaN
Name: country_code, dtype: object

In [15]:
#sostituisco il missing con il codice NA
loans["country_code"].fillna('NA', inplace=True)

In [17]:
#conto quante volte compare il lo stato
loans[["country_code"]]\
    .country_code\
    .value_counts()\
    .reset_index()

,index,country_code
0,PH,285336
1,KE,143699
2,PE,86000
3,KH,79701
4,SV,64037
...,...,...
91,PG,1
92,CA,1
93,UY,1
94,BW,1


5. For each country, compute the overall amount of money borrowed

In [18]:
#raggruppo per stato e sommo il campo loan_ampunt
amnt_borrow = loans.groupby("country_code")["loan_amount"].sum().reset_index()
amnt_borrow

,country_code,loan_amount
0,AF,1967950.0
1,AL,4307350.0
2,AM,22950475.0
3,AZ,14784625.0
4,BA,477250.0
...,...,...
91,XK,3083025.0
92,YE,3444000.0
93,ZA,1006525.0
94,ZM,1978975.0


6. Like the previous point, but expressed as a percentage of the overall amount lent

In [19]:
#calcolo la percentuale dividendo ogni loan_amount per il la somma di tutti (e moltiplico per 100)
amnt_borrow['percentage'] = amnt_borrow['loan_amount']/amnt_borrow['loan_amount'].sum()*100
amnt_borrow.sort_values('percentage', ascending=False)

,country_code,loan_amount,percentage
62,PH,97984600.0,8.293678
60,PE,79437775.0,6.723825
38,KE,66735975.0,5.648711
66,PY,53964700.0,4.567716
40,KH,51613525.0,4.368706
...,...,...,...
87,VI,10000.0,0.000846
89,VU,9250.0,0.000783
85,UY,8000.0,0.000677
12,BW,8000.0,0.000677


7. Like the three previous points, but split for each year (with respect to disburse time)

In [20]:
#.dropna().astype(int)
#aggiungo il campo year con solo l'anno preso da disburse_time
loans['year'] = loans['disburse_time'].dt.year

#raggruppo con country_code e anno e conto quante volte compaiono gli stati
loans.groupby([loans['country_code'],loans['year']])\
               ['country_code'].value_counts()

country_code  year    country_code
AF            2007.0  AF               408
              2008.0  AF               370
              2009.0  AF               678
              2010.0  AF               632
              2011.0  AF               247
                                      ... 
ZW            2013.0  ZW               426
              2014.0  ZW              2078
              2015.0  ZW               600
              2016.0  ZW               808
              2017.0  ZW              1079
Name: country_code, Length: 748, dtype: int64

In [21]:
#stesso di sopra ma sommo il loan_amount
loans.groupby([loans['country_code'],loans['year']])\
              ['loan_amount'].sum() 

country_code  year  
AF            2007.0     194975.0
              2008.0     365375.0
              2009.0     585125.0
              2010.0     563350.0
              2011.0     245125.0
                          ...    
ZW            2013.0     678525.0
              2014.0    1311575.0
              2015.0     723625.0
              2016.0     788600.0
              2017.0    1237600.0
Name: loan_amount, Length: 748, dtype: float64

In [22]:
#stesso ma calcolo la percentuale
loans.groupby([loans['country_code'],loans['year']])\
              ['loan_amount'].sum() / loans['loan_amount'].sum() *100

country_code  year  
AF            2007.0    0.016503
              2008.0    0.030926
              2009.0    0.049527
              2010.0    0.047683
              2011.0    0.020748
                          ...   
ZW            2013.0    0.057432
              2014.0    0.111015
              2015.0    0.061250
              2016.0    0.066749
              2017.0    0.104754
Name: loan_amount, Length: 748, dtype: float64

8. For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [23]:
#carico
lenders = pd.read_csv('../input/additional-kiva-snapshot/lenders.csv')

In [24]:
#calcolo il contributo dei lenders che fanno un certo loan semplicemente dividendo per il num_lenders_total
loans['lenders_contribute'] = loans['loan_amount'] / loans['num_lenders_total']

#unisco la tabella dei lenders con il contributo calcolato basandomi sul campo loan_id
lenders_contribute = pd.merge(df_lenders, 
                              loans[['loan_id','lenders_contribute']],
                              on = "loan_id")

In [25]:
#raggruppo tutti i lenders sommando i contribuiti dei vari loan
lenders_mnt = lenders_contribute.groupby('lenders', as_index=False)['lenders_contribute'].sum()
lenders_mnt

,lenders,lenders_contribute
0,000,1485.309656
1,00000,1249.947363
2,0002,2201.180463
3,00mike00,38.461538
4,0101craign0101,2424.088932
...,...,...
1639021,zzanita,62.500000
1639022,zzcyna7269,44.285714
1639023,zzinnia,32.758621
1639024,zzmcfate,2033.750197


9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [26]:
#controllo missing
lenders.isnull().sum()

permanent_name             0
display_name            2768
city                 1619306
state                1713481
country_code         1458635
member_since               0
occupation           1844514
loan_because         2174852
loan_purchase_num     894281
invited_by           1852349
num_invited                0
dtype: int64

In [27]:
#vedo quale sia la distribuzione dei valori di country_code contando quante volte appaiono
lenders['country_code'].value_counts()

US    591612
CA     67970
GB     38380
AU     37103
DE     16007
       ...  
IO         1
GW         1
NU         1
KM         1
TF         1
Name: country_code, Length: 234, dtype: int64

In [28]:
#fisso un seme
random.seed(123)

#salvo la distribuzione in una variabile
x = lenders['country_code'].value_counts(normalize=True)

#prendo i missing
missing = lenders['country_code'].isnull()
#sostituisco prendendo casualmente dalla distribuzione
lenders.loc[missing,'country_code'] = np.random.choice(x.index, size=len(lenders[missing]),p=x.values)

lenders

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,US,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,IN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,US,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,US,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,US,1461301154,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
2349169,janet7309,Janet,NaN,NaN,DE,1342097163,NaN,NaN,NaN,NaN,0
2349170,pj4198,NaN,NaN,NaN,US,1342097515,NaN,NaN,NaN,NaN,0
2349171,maria2141,Maria,NaN,NaN,US,1342099723,NaN,NaN,2.0,NaN,0
2349172,simone9846,Simone,NaN,NaN,US,1342100213,NaN,NaN,NaN,NaN,0


In [29]:
#rendo dataframe il dataset del punto ?
lenders_mnt=pd.DataFrame(lenders_mnt)
#normalizzo il campo lenders togliendo spazi che potrebbero compromettere un join
lenders_mnt.lenders = lenders_mnt.lenders.str.strip()

In [30]:
#unisco le tabelle con le informazioni sui lenders e su quanto abbiano contribuito
#con i campi permant_name e lenders
lenders_country = pd.merge(lenders[['permanent_name','country_code']],
                              lenders_mnt,
                              left_on='permanent_name',
                              right_on='lenders')
#elimino la colonna lenders che mi serviva per il join
lenders_country.drop('lenders', axis=1, inplace=True)

In [31]:
#raggruppo per coutry code e sommo i contributo di tutti i lenders
country_lent = lenders_country.groupby('country_code', as_index=False)['lenders_contribute'].sum()
country_lent

,country_code,lenders_contribute
0,AD,6.697064e+03
1,AE,1.853786e+06
2,AF,1.343690e+05
3,AG,1.724869e+03
4,AI,7.123247e+02
...,...,...
229,YE,4.154021e+04
230,YT,8.254812e+01
231,ZA,6.593791e+05
232,ZM,4.100530e+04


In [32]:
#raggruppo per coutry code e sommo l'ammontare di tutti i loan
country_borrowed = loans.groupby(['country_code'], as_index=False)['loan_amount'].sum()
country_borrowed

,country_code,loan_amount
0,AF,1967950.0
1,AL,4307350.0
2,AM,22950475.0
3,AZ,14784625.0
4,BA,477250.0
...,...,...
91,XK,3083025.0
92,YE,3444000.0
93,ZA,1006525.0
94,ZM,1978975.0


In [33]:
#unisco le due tabelle con country code
borrow_lent = pd.merge(country_lent, country_borrowed, how='outer', on = 'country_code')

#rinomino
borrow_lent.columns = ['country_code', 'money_lent','money_borrowed']

#riempio i dati mancanti con 0 (che corrispondo a una mancata presenza di un debito)
borrow_lent.fillna(0, inplace=True)

In [34]:
#calcolo la differenza tra quanto ho prestato e quanto devo
borrow_lent['difference'] = borrow_lent['money_lent'] -\
                            borrow_lent['money_borrowed']
#sortisco in ordine descrescente
borrow_lent.sort_values('difference', ascending=False)

,country_code,money_lent,money_borrowed,difference
216,US,6.360434e+08,46352000.0,5.896914e+08
35,CA,8.445636e+07,50000.0,8.440636e+07
13,AU,5.515254e+07,0.0,5.515254e+07
72,GB,5.275106e+07,0.0,5.275106e+07
52,DE,3.723174e+07,0.0,3.723174e+07
...,...,...,...,...
108,KH,1.076310e+05,51613525.0,-5.150589e+07
172,PY,5.321158e+04,53964700.0,-5.391149e+07
106,KE,8.519194e+05,66735975.0,-6.588406e+07
161,PE,1.703891e+05,79437775.0,-7.926739e+07


10. Which country has the highest ratio between the difference computed at the previous point and the population?

In [35]:
#carico
country_stats = pd.read_csv("../input/additional-kiva-snapshot/country_stats.csv")
country_stats

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Somalia,SO,SOM,Africa,Eastern Africa,14742523,NaN,NaN,NaN,NaN,NaN,NaN,Somalia
170,Central African Republic,CF,CAF,Africa,Middle Africa,4659080,NaN,0.352440,51.458,7.098980,4.230000,587.473961,Central African Republic
171,Samoa,WS,WSM,Oceania,Polynesia,196440,NaN,0.702000,NaN,NaN,NaN,NaN,Samoa
172,Palestine,PS,PS,Asia,Western Asia,4920724,NaN,0.677000,NaN,NaN,NaN,NaN,Palestine


In [36]:
#controllo missing
country_stats.isnull().sum()

country_name                      0
country_code                      1
country_code3                     1
continent                         0
region                            0
population                        0
population_below_poverty_line    22
hdi                               3
life_expectancy                   6
expected_years_of_schooling       6
mean_years_of_schooling           6
gni                               6
kiva_country_name                 0
dtype: int64

In [37]:
#guardo chi ha il country code missing
country_stats[country_stats[['country_code']].isna().any(axis=1)][['country_name', 'country_code']]

,country_name,country_code
115,Namibia,NaN


In [38]:
#come prima lo fillo con NA
country_stats["country_code"].fillna('NA', inplace=True)

In [39]:
#calcolo chi è povero
country_stats['population_not_poor'] = country_stats['population'] - \
                                      (country_stats['population_below_poverty_line'] * country_stats['population']/100)


In [40]:
#unisco i dataset con tutte le info che mi servono per i due ratio
ratio = pd.merge(country_stats[['country_code','population','population_not_poor']],
                 borrow_lent[['country_code','difference']],
                 on='country_code')

In [41]:
#calcolo il primo ratio, e ordino
ratio['ratio1']=ratio['difference']/ratio['population']
ratio=ratio.sort_values(by='ratio1', ascending=False)
ratio[['country_code','ratio1']]

,country_code,ratio1
161,NO,3.502564
72,CA,2.304661
150,AU,2.255676
156,IS,1.923771
6,US,1.817458
...,...,...
119,MN,-4.964837
82,SV,-6.530205
109,AM,-7.807932
93,PY,-7.915011


11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line

In [42]:
#calcolo il secondo ratio e ordino
ratio['ratio2']=ratio['difference']/ratio['population_not_poor']
ratio=ratio.sort_values(by='ratio2', ascending=False)
ratio[['country_code','ratio2']]

,country_code,ratio2
72,CA,2.543776
6,US,2.140704
96,SE,1.781450
95,NL,1.590600
122,CH,1.270346
...,...,...
168,SO,NaN
163,PR,NaN
167,SB,NaN
170,PS,NaN



12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD 31 / (31+365+30) = 363.85 for 2016, 5000USD 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018

In [43]:
#guardo il range temporale massimo di tutti glia nni presenti 
c = loans['disburse_time'].dt.year.unique()
d = loans['planned_expiration_time'].dt.year.unique()
c.sort()
d.sort()
print(c)
print(d)

[2005. 2006. 2007. 2008. 2009. 2010. 2011. 2012. 2013. 2014. 2015. 2016.
 2017. 2018.   nan]
[2012. 2013. 2014. 2015. 2016. 2017. 2018.   nan]


In [44]:
#creo una variabile bisestile che racchiude gli anni bisestili
bisestile = {2008, 2012, 2016}

In [45]:
#loans = loans.dropna()
#loans['year'] = loans['year'].astype(int)

In [46]:
#creo le variabli che mi serviranno per il calcolo prendendo le informazioni dalla tabella loan
# anno di inizio per ogni loan
# anno di fine per ogni loan
# l'ammontare del loan
# la durata espressa in giorni del loan

inizio  = [inizio.year for inizio in loans['disburse_time']]
fine    = [fine.year   for fine   in loans['planned_expiration_time']]
amounts = list(loans['loan_amount'])
durata  = loans['duration']

testo il codice con l'esempio

In [47]:
#0: creo le liste che mi serviranno
lanno=[]
limporto=[]
#salvo il range degli anni in una variabile
anni=list(range(2016,2018+1))    

#1
# prendo il primo anno e controllo se è bisestile se lo è i giorni saranno 366
# calcolo quanti giorni mancano alla fine dell'anno:  
# 365 - data_inizio (dove la data di inizio è espressa con il numero del giorno dell'anno)
# poi spalmo l'importo come descritto day*importo/durata totale
# e salvo nelle liste ausiliare anno e importo spalmato 
if 2016 in bisestile:
    ga=366-(pd.to_datetime('2016-12-01').dayofyear-1)
else:    
    ga=365-(pd.to_datetime('2016-12-01').dayofyear-1)
ia=(ga*5000)/(31+365+30)
lanno.append(anni[0])
limporto.append(round(ia,2))

#2  
# prendo tutti gli "anni di mezzo", cioè che non sono quello di inizio o fine
# guardo se è bisestile, in quel caso 366
# calcolo l'importo spalmandolo su tutto l'anno
# salvo nelle liste ogni anno e importo
for j in range(1,len(anni)-1):
    if anni[j] in bisestile:
        ia=(366*5000)/(31+365+30)
    else:
        ia=(365*5000)/(31+365+30)
    lanno.append(anni[j])
    limporto.append(round(ia,2))
    
#3
# considero l'anno di fine
# trovo a che giorno siamo sempre con la funzione dayofyear
# spalmo l'importo 
# salvo
ga=pd.to_datetime('2018-01-30').dayofyear
ia=(ga*5000)/(31+365+30)
lanno.append(anni[-1])
limporto.append(round(ia,2))

In [48]:
#costruisco il dataframe dalle liste salvate 
esercizio = pd.DataFrame()
esercizio['Anno']    = lanno
esercizio['Importo'] = limporto
#aggiungo la colonna con le soluzioni fornite 
esercizio['Confronto'] = (363.85,4284.04,352.11)
#confronto
if esercizio['Confronto'].all() == esercizio['Importo'].all():
    print('OK')

OK


In [49]:
esercizio

,Anno,Importo,Confronto
0,2016,363.85,363.85
1,2017,4284.04,4284.04
2,2018,352.11,352.11


In [ ]:
# ri inizializzo le liste
lanno   =[]
limporto=[]

#apro un ciclo che scorre su tutti i loan
for i in range(len(loans)):
    #calcolo solo quelli "sensati" che hanno inizio prima della fine
    if durata[i].days >= 0:
        #considero il caso in cui inizia e finisca lo stesso anno
        #e in quel caso l'importo andrà tutto in quell'anno
        if inizio[i]==fine[i]:
            lanno.append(inizio[i])
            limporto.append(amounts[i])
        else:
            #come prima
            #0
            anni=list(range(inizio[i],fine[i]+1)) 
            
            #1
            if anni[0] in bisestile:
                giorni=366-(loans['disburse_time'][i].dayofyear-1)
            else:    
                giorni=365-(loans['disburse_time'][i].dayofyear-1)
            importo=(giorni*amounts[i])/durata[i].days
            lanno.append(anni[0])
            limporto.append(importo)
            
            #2
            for j in range(1,len(anni)-1):
                if anni[j] in bisestile:
                    importo=(366*amounts[i])/durata[i].days
                else:
                    importo=(365*amounts[i])/durata[i].days
                lanno.append(anni[j])
                limporto.append(importo)

            #3
            giorni=loans['planned_expiration_time'][i].dayofyear
            importo=(giorni*amounts[i])/durata[i].days
            lanno.append(anni[-1])
            limporto.append(importo)

In [ ]:
#creo un dataset 
anno_importo            = pd.DataFrame()
anno_importo['Anno']    = lanno
anno_importo['Importo'] = limporto

In [ ]:
#gruppo per anno e sommo l'importo (e forzo a due decimali)
pd.DataFrame(anno_importo\
                 .groupby('Anno')['Importo']\
                 .sum()
            )\
  .style.format("{:.2f}")